# 对象持久化

如果我们希望传输或者保存python中的对象,就需要用到对象持久化技术.这通常会用在分布式系统上.python的对象持久化方案目前通用且主流的有如下几种:

+ `pickle`
+ `dill`
+ `cloudpickle`


## pickle用于持久化python的内置简单对象

pickle是标准库中的模块,它实现了最基本的数据对象持久化能力.通过pickle模块的序列化操作我们能够将程序中运行的对象信息保存到文件中去,永久存储;通过pickle模块的反序列化操作,我们能够从文件中创建上一次程序保存的对象.

需要注意,pickel的文件并不是默认跨版本支持的,可以对照这张表设定需要的参数

pickel到目前为止有5种序列化格式:

版本|说明|支持python版本
---|---|---
0|人类可读的文本,用于最早期|全部版本
1|老的二进制版本文本同样用于早期|全部版本
2|出现于2.3版本,用以支持新类|2.3+
3|出现于3.0版本,用以支持bytes类型|3.0+
4|出现于Python 3.4.用于扩充pickel的支持类型和大对象|3.4+

python3.5+默认使用的是版本4的pickle格式

要指定使用某一种pickle格式,可以在方法中使用`protocol:int=n`来指定

### pickle的局限性

pickle的兼容性问题一直让人诟病,除了python没有别的语言使用pickle,而如上表所示,pickle在各个版本的python中也不是默认通用的

pickle实际上并不能传递函数或者类,而是只能记录下它的状态信息而已,因此不能跨模块传递,除此之外,一些对象类型也是不可pickle的.例如Python不能 pickle文件对象(或者任何带有对文件对象引用的对象),因为 Python 在 unpickle 时不能保证它可以重建该文件的状态.


### 使用示例

In [2]:
import pickle
exa_l=[1,2,3,4,5]
exa_b = pickle.dumps(exa_l)
exa_b

b'\x80\x04\x95\x0f\x00\x00\x00\x00\x00\x00\x00]\x94(K\x01K\x02K\x03K\x04K\x05e.'

In [3]:
pickle.loads(exa_b)

[1, 2, 3, 4, 5]

In [4]:
with open("source/pickle_test.txt","wb") as f:
    pickle.dump(exa_l,f)
with open("source/pickle_test.txt","rb") as f:
    view_exam = pickle.load(f)
view_exam

[1, 2, 3, 4, 5]

### 命令行工具 `pickletools`

在python3中提供了一个命令行工具来管理pickle文件

可用的参数:

+ `-a`,`–annotate`: 用简短的操作码描述来标注每一行
+ `-o`,`–output`: 写入输出的文件的名称.
+ `-l`,`–indentlevel`:用于缩进新的MARK级别的空白数
+ `-m`,`–memo`:拆卸多个物体时,请在拆卸之间保留备注.
+ `-p`,`–preamble`:当指定多于一个pickle文件时,在每次分解之前打印给定的前导码

In [6]:
!python -m pickle source/pickle_test.txt

[1, 2, 3, 4, 5]


In [7]:
!python -m pickletools source/pickle_test.txt

    0: \x80 PROTO      4
    2: \x95 FRAME      15
   11: ]    EMPTY_LIST
   12: \x94 MEMOIZE    (as 0)
   13: (    MARK
   14: K        BININT1    1
   16: K        BININT1    2
   18: K        BININT1    3
   20: K        BININT1    4
   22: K        BININT1    5
   24: e        APPENDS    (MARK at 13)
   25: .    STOP
highest protocol among opcodes = 4


## [dill](https://github.com/uqfoundation/dill)用于序列化python对象*

dill支持几乎所有的python数据,按github上的说法,它支持:

none, type, bool, int, long, float, complex, str, unicode,
tuple, list, dict, file, buffer, builtin,
both old and new style classes,
instances of old and new style classes,
set, frozenset, array, functions, exceptions
functions with yields, nested functions, lambdas
cell, method, unboundmethod, module, code, methodwrapper,
dictproxy, methoddescriptor, getsetdescriptor, memberdescriptor,
wrapperdescriptor, xrange, slice,
notimplemented, ellipsis, quit


还不支持的有:

frame(帧),generator(生成器对象,因为包含帧状态),traceback(依然是因为无法保存帧状态)


与pickle不同,dill的序列化可以跨模块传递,事实上dill也是为了分布式计算传递python对象而设计的.

dill对python3.5+支持不错,它支持协程,也支持numpy数组,只是序列化的过程中typehint会被消除.



dill的设计目标是为分布式系统传输对象提供支持.因此支持的类型最多,但实际使用它的时候由于它序列化后的字节长度往往过大所以可能反而并不适合用于传输,而它的全面细致让它在用于保存时其实更有优势.

dill在windows上似乎并不是完全支持.因此跨平台使用时需要谨慎.


dill可以直接使用pip安装,使用也相当简单,只要替代pickle就行了,他们接口相同

In [22]:
import dill

In [23]:
from asyncio import sleep
import asyncio
async def asyng(n:int):
    print("n:"+str(n)+" wait")
    await sleep(1)
    print("n:"+str(n)+"done")

In [24]:
with open("source/dill_cor.txt","wb") as f:
    dill.dump(asyng,f)

In [25]:
with open("source/dill_cor.txt","rb") as f:
    s1 = dill.load(f)

In [26]:
loop = asyncio.new_event_loop()

In [27]:
loop.run_until_complete(asyncio.wait([s1(1),s1(2),s1(3)]))

n:2 wait
n:3 wait
n:1 wait
n:2done
n:3done
n:1done


({<Task finished coro=<asyng() done, defined at <ipython-input-23-badfb7211783>:3> result=None>,
  <Task finished coro=<asyng() done, defined at <ipython-input-23-badfb7211783>:3> result=None>,
  <Task finished coro=<asyng() done, defined at <ipython-input-23-badfb7211783>:3> result=None>},
 set())

## *[cloudpickle](https://github.com/cloudpipe/cloudpickle)另一个pickle序列化工具*

cloudpickle的开发目的也是一样而且现在已经在pyspark和dask中实用(dask中之前使用的是dill),它相较于dill,cloudpickle更加健壮,从目前来看应该是更加适合用于传输的工具.实际上它只是实现了序列化,而反序列化是交给自带的pickle的,这种设计可以在反序列化一端减少依赖,也让反序列化更加快速.

从指标上来说:

+ cloudpickle 序列化更加快速,比dill快大约10%
+ cloudpickle 反序列化更加快速,比dill块大约85%
+ cloudpickle 序列化出的bytes长度大约比dill长20%(带typehints的情况)
+ cloudpickle 支持typehits


### 序列化

In [28]:
class A:
    test:str
    def __init__(self,text:str)->None:
        self.text = text

In [29]:
import cloudpickle

In [30]:
AP = cloudpickle.dumps(A)

### 反序列化

In [31]:
import pickle

In [32]:
AO = pickle.loads(AP)

In [33]:
AO.__annotations__

{'test': str}